# Creating a SVM to predict the probabilities a team has of winning

## Importing the data

In [3]:
import numpy as np
from sklearn.svm import SVC

In [4]:
dataset = np.load('dataset.npy', allow_pickle=True)
classes = np.load('classifications.npy', allow_pickle=True)

In [13]:
print(dataset.shape)
print(classes.shape)

(53013, 350)
(53013,)


## Creating the SVM

In [19]:
SVM = SVC(gamma='auto', probability=True)  # use standard parameters for now
SVM.fit(dataset, classes)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

For testing purposes, lets create a vector with random heroes/bans and a random map and lets see the result

In [20]:
test_vector = np.zeros(350)

It needs to have 5 1's in the first 84 positions, 3 1's in the second 84 positions, then 5 in the next 84 and then 3 in the next 84. Finally, only one 1 in the last 14 positions.

In [21]:
test_vector[10] = 1
test_vector[20] = 1
test_vector[30] = 1
test_vector[40] = 1
test_vector[50] = 1
test_vector[90] = 1
test_vector[100] = 1
test_vector[110] = 1
test_vector[180] = 1
test_vector[190] = 1
test_vector[200] = 1
test_vector[210] = 1
test_vector[220] = 1
test_vector[290] = 1
test_vector[300] = 1
test_vector[310] = 1
test_vector[340] = 1

In [26]:
SVM.probB_

array([0.08764727])